In [161]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [162]:
#localidades = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Localidades.csv",delimiter = ',',encoding = "utf-8")
#clientes = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Clientes.csv",delimiter = ';',encoding = "utf-8")
#clientes = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Clientes.csv",encoding = "utf-8",delimiter = ';', usecols=["ID","Provincia","Nombre_y_Apellido","Domicilio","Telefono","Edad","Localidad","X","Y","col10"])
#proveedores = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Proveedores.csv",delimiter = ',',encoding = "ansi")
gasto = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Gasto.csv",delimiter = ',',encoding = "utf-8")
#compra = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Compra.csv",delimiter = ',',encoding = "utf-8")
sucursales = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Sucursales.csv",delimiter = ';',encoding = "utf-8")
#venta = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Venta.csv",delimiter = ',',encoding = "utf-8")

CARGA Y TRATAMIENTO DE TABLA VENTAS 

In [163]:
def cargadeVentas():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Vent*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    venta=pd.concat(df_list)
    #Tratamiento de Outliers de Precio 
    Q1 = venta["Precio"].quantile(0.25)
    Q3 = venta["Precio"].quantile(0.75)
    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR)) 
    mask = venta["Precio"]<outliersSup
    ventaSinOut2 = venta[mask]
    #Creo columnar de Outliers
    venta["Outlier"]=np.ones(venta.shape[0])
    #Marco Outliers
    venta["Outlier"].loc[(venta['Precio'] >= outliersSup)] = 0
    #Se crea tabla con precio prodemio por tipo de producto
    Precio_Producto= ventaSinOut2.groupby(["IdProducto"])["Precio"].mean().reset_index()
    Precio_Producto
    venta2=venta.copy()
    #Reemplazamos valores nulos en Precio y Cantidad
    venta2.Precio.fillna(0, inplace=True)
    venta2.Cantidad.fillna(1, inplace=True)
    Precio_Producto= Precio_Producto.rename(columns= {"Precio":"Precio2"})
    #Agregamos columna auxiliar para obtener valor medio por tipo de producto 
    venta2 = pd.merge(venta2,Precio_Producto, how="left", on=["IdProducto"])
    #Ordeno por IdDeVenta
    venta2.sort_values("IdVenta")
    #Reemplazamos ventas con precio 0 u Outliers con precios promedio de tipo de producto
    #venta2["Precio"]=venta2.apply(lambda x: x["Precio2"] if x["Precio"]==0  else x["Precio"], axis=1)
    for i in range(0,venta2.shape[0]):
        if venta2.Precio[i] == 0 or venta2.Precio[i]>outliersSup:
            venta2.iloc[i,8]=venta2.iloc[i,11] # 8 es precio, 10 es precio 2
    venta2.drop(columns="Precio2", inplace=True)
    #Tratamiento de Outlier de Cantidad 
    Q1 = venta2["Cantidad"].quantile(0.25)
    Q3 = venta2["Cantidad"].quantile(0.75)
    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR)) 
    mask = venta2["Cantidad"]<outliersSup
    ventaCantSinOut = venta2[mask]
    cantidadMedia = round(ventaCantSinOut.Cantidad.mean())
    venta2.loc[(venta2['Cantidad'] >= outliersSup),"Cantidad"] = cantidadMedia
    #Se crea columna de Monto de Venta
    venta2["Monto"]= venta2["Precio"] * venta2["Cantidad"]*venta2["Outlier"]
    venta2.to_csv (r'C:\Users\andre\OneDrive\Escritorio\TP Individual N1\DS-PI-ProyectoIndividual\datasetsProcesados\ventas_dataframe.csv', index = False, header=True)

In [164]:
cargadeVentas()

C:\Users\andre\AppData\Local\Temp\ipykernel_10940\1203019098.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venta["Outlier"].loc[(venta['Precio'] >= outliersSup)] = 0


CARGA Y TRATAMIENTO DE TABLA COMPRAS 

In [165]:
def cargadeCompras():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Compra.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    compra=pd.concat(df_list)
    Q1 = compra["Precio"].quantile(0.25)
    Q3 = compra["Precio"].quantile(0.75)
    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR)) 

    mask = compra["Precio"]<outliersSup
    compraSinOut2 = compra[mask]

    Precio_Producto= compraSinOut2.groupby(["IdProducto"])["Precio"].mean().reset_index()
    Precio_Producto= Precio_Producto.rename(columns= {"Precio":"Precio2"})

    compra2=compra.copy()
    compra2 = pd.merge(compra2,Precio_Producto, how="left", on=["IdProducto"])
    compra2.sort_values("IdCompra")

    #se reemplazan los 0 de precio2 por el valor de la media de los precios sin outliers.
    compra2.Precio2.fillna(compraSinOut2.Precio.mean(),inplace=True)

    #se reemplazan los nan por 0 en la columna precio.
    compra2.Precio.fillna(0,inplace=True)
    #se insertan 1 cuando la cantidad es 0.
    compra2.Cantidad.fillna(1,inplace=True)
    #Se crea una columna con unos (1) 
    compra2["Outliers"]=np.ones(compra2.shape[0])
    #se marcan con 0 las filas que son outlaiers, para luego ser reemplazadas por Precio2
    compra2.loc[(compra2['Precio'] >= outliersSup),"Outliers"] = 0

    #Este bucle reemplaza los ceros y outliers por los valores promedios 
    for i in range(0,compra2.shape[0]):
      if compra2.Precio[i] == 0 or compra2.Precio[i]>outliersSup:
        compra2.iloc[i,7]=compra2.iloc[i,9] # 7 es precio, 9 es precio 2
        
    #se elimina la columna Precio2
    compra2.drop(columns="Precio2", inplace=True)
    #Se crea la columna Total
    #compra2["Total"]= compra2["Precio"]*compra2["Cantidad"]
    #Se calcula el rango intercuartilico IQR.
    Q1 = compra2["Cantidad"].quantile(0.25)
    Q3 = compra2["Cantidad"].quantile(0.75)
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR)) 
    # se crea un df sin outliers, se calcula la media de compras,y se imputan como valor en los casos que sean outliers.
    mask = compra2["Cantidad"]<outliersSup
    compraCantSinOut = compra[mask]
    cantidadMedia = round(compraCantSinOut.Cantidad.mean())
    compra2.loc[(compra2['Cantidad'] >= outliersSup),"Cantidad"] = cantidadMedia #8
    compra2["Total"]= compra2["Precio"]*compra2["Cantidad"]
    compra2.to_csv (r'C:\Users\andre\OneDrive\Escritorio\TP Individual N1\DS-PI-ProyectoIndividual\datasetsProcesados\compras_dataframe.csv', index = False, header=True)


In [166]:
cargadeCompras()

CARGA Y TRATAMIENTO DE CLIENTES

In [167]:
def cargadeLocalidades():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Localidades.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    global localidades
    
    localidades=pd.concat(df_list)

    localidades["nombre"]=localidades.apply(lambda x: "Ciudad de Buenos Aires" if x["municipio_nombre"]=="Capital Federal"  else x["nombre"], axis=1)
    localidades.drop(['categoria','fuente', 'localidad_censal_id', 'localidad_censal_nombre', 'municipio_id', 'municipio_nombre' ], axis = 'columns', inplace=True)
    localidades["departamento_id"].fillna("Sin datos", inplace=True)
    localidades["departamento_nombre"].fillna("Sin datos", inplace=True)
    localidades= localidades.rename(columns= {"centroide_lat":"Latitud"})
    localidades= localidades.rename(columns= {"centroide_lon":"Longitud"})
    localidades_repetidas =localidades.id.value_counts()
    localidades_repetidas[localidades_repetidas>1]

    #Creacion de df Provincia
    provincias = pd.DataFrame()
    provincias["Provincia"] = localidades.provincia_nombre
    provincias["IdProvincia"]=localidades.provincia_id

    provincias= provincias.drop_duplicates()
    departamentos= pd.DataFrame()
    departamentos["departamento_nombre"] = localidades.departamento_nombre
    departamentos["departamento_id"]=localidades.departamento_id
    departamentos= departamentos.rename(columns= {"departamento_id":"IdDepartamento","departamento_nombre":"Departamento"})
    departamentos= departamentos.drop_duplicates()
    localidades.drop(columns="departamento_nombre", inplace=True)


    localidades= localidades.rename(columns= {"provincia_nombre":"Provincia"})
    localidades["nombre"]= localidades["nombre"].str.title()
    localidades["Provincia"]= localidades["Provincia"].str.title()
    localidades= localidades.rename(columns= {"id":"IdLocalidad", "nombre":"Localidad","provincia_id":"IdProvincia", "departamento_id":"IdDepartamento"})

    #Borroduplicados
    localidades.drop_duplicates(subset=["Localidad","IdProvincia"], inplace=True)
    localidades.reset_index(inplace=True)
    
    localidades["Provincia"]=[x.replace('á',"a") for x in localidades["Provincia"]]
    localidades["Provincia"]=[x.replace('é',"e") for x in localidades["Provincia"]]
    localidades["Provincia"]=[x.replace('í',"i") for x in localidades["Provincia"]]
    localidades["Provincia"]=[x.replace('ó',"o") for x in localidades["Provincia"]]
    localidades["Provincia"]=[x.replace('ú',"u") for x in localidades["Provincia"]]
    localidades["Localidad"]=[x.replace('á',"a") for x in localidades["Localidad"]]
    localidades["Localidad"]=[x.replace('é',"e") for x in localidades["Localidad"]]
    localidades["Localidad"]=[x.replace('í',"i") for x in localidades["Localidad"]]
    localidades["Localidad"]=[x.replace('ó',"o") for x in localidades["Localidad"]]
    localidades["Localidad"]=[x.replace('ú',"u") for x in localidades["Localidad"]]
    
    
    localidades=localidades.append({'Latitud' : 0 , 'Longitud' : 0, 'IdLocalidad' : 0, 'Localidad':"Sin Datos","Provincia":"Sin Datos"} , ignore_index=True)
    localidades.to_csv (r'C:\Users\andre\OneDrive\Escritorio\TP Individual N1\DS-PI-ProyectoIndividual\datasetsProcesados\localidades_dataframe.csv', index = False, header=True)
    provincias.to_csv (r'C:\Users\andre\OneDrive\Escritorio\TP Individual N1\DS-PI-ProyectoIndividual\datasetsProcesados\provincias_dataframe.csv', index = False, header=True)
    departamentos.to_csv (r'C:\Users\andre\OneDrive\Escritorio\TP Individual N1\DS-PI-ProyectoIndividual\datasetsProcesados\departamentos_dataframe.csv', index = False, header=True)
    

In [168]:
cargadeLocalidades()

C:\Users\andre\AppData\Local\Temp\ipykernel_10940\661035760.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  localidades=localidades.append({'Latitud' : 0 , 'Longitud' : 0, 'IdLocalidad' : 0, 'Localidad':"Sin Datos","Provincia":"Sin Datos"} , ignore_index=True)


CARGA Y TRATAMIENTO DE CLIENTES

In [169]:
def cargadeClientes():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Client*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ';',encoding = "utf-8", usecols=["ID","Provincia","Nombre_y_Apellido","Domicilio","Telefono","Edad","Localidad","X","Y","col10"])
        df_list.append(df)
    clientes=pd.concat(df_list)
    clientes.drop(columns="col10", inplace=True)
    #Se corrigen nombre de Columnas 
    clientes= clientes.rename(columns= {"ID":"IdCliente"})
    clientes= clientes.rename(columns= {"X":"Longitud"})
    clientes= clientes.rename(columns= {"Y":"Latitud"})

    #Letra capital
    clientes["Nombre_y_Apellido"]= clientes["Nombre_y_Apellido"].str.title()
    clientes["Domicilio"]= clientes["Domicilio"].str.title()
    clientes["Localidad"]= clientes["Localidad"].str.title()

    #Reemplazo de datos faltantes
    clientes.Provincia.fillna("Sin Datos", inplace=True)
    clientes.Nombre_y_Apellido.fillna("Sin Datos", inplace=True)
    clientes.Localidad.fillna("Sin Datos", inplace=True)

    clientes.Telefono.fillna("Sin Datos", inplace=True)
    clientes.Domicilio.fillna("Sin Datos", inplace=True)

    #Reemplazo , por . 
    clientes["Latitud"] = clientes["Latitud"].str.replace(",", ".")
    clientes["Longitud"] = clientes["Longitud"].str.replace(",", ".")


    #Reemplazo Latitud y Longitud nulos  con 0
    clientes.Latitud.fillna(0.0, inplace=True)
    clientes.Longitud.fillna(0.0, inplace=True)
    #Normalizo provincia de Buenos Aires 
    clientes["Provincia"]=clientes.apply(lambda x: "Buenos Aires" if x["Provincia"]=="Ciudad de Buenos Aires"  else x["Provincia"], axis=1)
    #Tratamiendo de acentos 
    clientes["Provincia"]=[x.replace('á',"a") for x in clientes["Provincia"]]
    clientes["Provincia"]=[x.replace('é',"e") for x in clientes["Provincia"]]
    clientes["Provincia"]=[x.replace('í',"i") for x in clientes["Provincia"]]
    clientes["Provincia"]=[x.replace('ó',"o") for x in clientes["Provincia"]]
    clientes["Provincia"]=[x.replace('ú',"u") for x in clientes["Provincia"]]
    clientes["Localidad"]=[x.replace('á',"a") for x in clientes["Localidad"]]
    clientes["Localidad"]=[x.replace('é',"e") for x in clientes["Localidad"]]
    clientes["Localidad"]=[x.replace('í',"i") for x in clientes["Localidad"]]
    clientes["Localidad"]=[x.replace('ó',"o") for x in clientes["Localidad"]]
    clientes["Localidad"]=[x.replace('ú',"u") for x in clientes["Localidad"]]
    #Merge entre df Localidades y Clientes para traer dato IdLocalidad a Clientes 
    clientes = pd.merge(clientes,localidades, how="left", on=["Provincia","Localidad"])

    #Reemplazo Latitud y Longitud sin datos con Latitud y Longitud de tabla Localidades 
    clientes["Longitud_x"]=clientes.apply(lambda x: x["Longitud_y"] if x["Longitud_x"]==0  else x["Longitud_x"], axis=1)
    clientes["Latitud_x"]=clientes.apply(lambda x: x["Latitud_y"] if x["Latitud_x"]==0  else x["Latitud_x"], axis=1)
    #Renombro columnas
    clientes= clientes.rename(columns= {"Longitud_x":"Longitud"})
    clientes= clientes.rename(columns= {"Latitud_x":"Latitud"})

    #Cambio de tipo de dato de Latitud y Longitud
    clientes["Latitud"]= clientes.Latitud.astype(float)
    clientes["Longitud"]= clientes.Longitud.astype(float)
    #Elimino columnas que se agregaron en el merge
    clientes.drop(['index'], axis = 'columns', inplace=True)
    #Funcion para encontrar Localidades por similitud - Se recupera IdLocalidad y IdProvincia
    def verificarLocalidadClientes(row):
        ratiomayor=0
        for i in range(0,localidades.shape[0]): 

                Ratio= fuzz.token_sort_ratio(row["Localidad"].lower(), localidades.Localidad[i].lower())
                if Ratio>ratiomayor:
                        ratiomayor=Ratio
                        #venta2.iloc[i,8]
                        #miloc= localidades.iloc[i,4]
                        miloc=localidades.Localidad[i]
                        #id=localidades.iloc[i,3]
                        id=localidades.IdLocalidad[i]
                        #idprov=localidades.iloc[i,5]
                        idprov=localidades.IdProvincia[i]

        row["Localidad"]=miloc
        row["IdLocalidad"]=id
        row["IdProvincia"]=idprov


        return row 
    #Creo DF de clientes con IdLocalidad faltante 
    clientes_nulos= clientes.loc[clientes['IdLocalidad'].isnull() == True].copy()
    #Aplico funcion
    clientes_nulos= clientes_nulos.apply(verificarLocalidadClientes,axis=1)
    #Borro columnas innecesarias para el merge 
    clientes_nulos.drop(columns=["Latitud","Longitud"],inplace=True)

    #Merge de Clientes con DF Clientes con IDlocalidad Nulos (ya corregido)
    clientes = pd.merge(clientes,clientes_nulos, how="left", on=["IdCliente"])

    #Completo IdLocalidad en tabla clientes original con 0 para prepararlos para el reemplazo
    clientes["IdLocalidad_x"]= clientes["IdLocalidad_x"].fillna(0)
    clientes["IdProvincia_x"]= clientes["IdProvincia_x"].fillna(0)
    #Realizo reemplazo de IdLocalidad y IdProvincia traidos con el merge
    clientes["IdLocalidad_x"]=clientes.apply(lambda x: x["IdLocalidad_y"] if x["IdLocalidad_x"]==0  else x["IdLocalidad_x"], axis=1)
    clientes["IdProvincia_x"]=clientes.apply(lambda x: x["IdProvincia_y"] if x["IdProvincia_x"]==0  else x["IdProvincia_x"], axis=1)


    #25 Localidades sin IdLocalidad - se lo completa como sin dato
    clientes["IdLocalidad_x"]=clientes["IdLocalidad_x"].fillna(0)

    #Elimino columnas innecesarias
    clientes.drop(['Latitud_y_x','Longitud_y_x','IdDepartamento_x','Provincia_y','Localidad_y', 'Nombre_y_Apellido_y'], axis = 'columns', inplace=True)
    clientes.drop(['IdProvincia_y','IdLocalidad_y','IdDepartamento_y','Longitud_y_y','Latitud_y_y', 'Edad_y', 'Telefono_y', 'Domicilio_y'], axis = 'columns', inplace=True)
    #Renombro columnas
    clientes= clientes.rename(columns= {"Provincia_x":"Provincia"})
    clientes= clientes.rename(columns= {"Nombre_y_Apellido_x":"Nombre_y_Apellido"})
    clientes= clientes.rename(columns= {"Localidad_x":"Localidad"})
    clientes= clientes.rename(columns= {"Longitud_x":"Longitud"})
    clientes= clientes.rename(columns= {"Latitud_x":"Latitud"})
    clientes= clientes.rename(columns= {"IdLocalidad_x":"IdLocalidad"})
    clientes= clientes.rename(columns= {"IdProvincia_x":"IdProvincia"})
    clientes.to_csv (r'C:\Users\andre\OneDrive\Escritorio\TP Individual N1\DS-PI-ProyectoIndividual\datasetsProcesados\clientes_dataframe.csv', index = False, header=True)

In [170]:
cargadeClientes()

PROVEEDORES CARGA Y TRATAMIENTO

In [171]:
proveedores = pd.read_csv("DS-PI-ProyectoIndividual\Datasets\Proveedores.csv",delimiter = ',',encoding = "ansi")
proveedores.fillna("Sin Dato")
proveedores= proveedores.rename(columns= {"IDProveedor":"IdProveedor", "Address":"Direccion","City":"Ciudad","State":"Provincia","Country":"Pais","departamen":"Departamento"})
proveedores.to_csv (r'C:\Users\andre\OneDrive\Escritorio\TP Individual N1\DS-PI-ProyectoIndividual\datasetsProcesados\proveedores_dataframe.csv', index = False, header=True)